# Ontology evaluation using SPARQL queries and competency questions


In [1]:
import rdflib
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
from rdflib import Namespace
from rdflib import URIRef
from owlrl import DeductiveClosure, OWLRL_Semantics


In [2]:
newg = rdflib.Graph() # we create a new graph
newg.parse("etvcontology.ttl", format="ttl")
DeductiveClosure(OWLRL_Semantics).expand(newg)
num_triples = len(newg)
print(f"The graph contains {num_triples} triples.")
for s, p, o in newg:
    print (s, p, o)


The graph contains 768 triples.
http://www.w3.org/2001/XMLSchema#integer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2001/XMLSchema#dateTime http://www.w3.org/2002/07/owl#sameAs http://www.w3.org/2001/XMLSchema#dateTime
http://www.w3.org/2002/07/owl#Nothing http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.W3C.org/casaz/ontologies/2024/10/etvco/Object
n739643f02e8c4793b820cbbccecbb031b1 http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.W3C.org/casaz/ontologies/2024/10/etvco/Concept
http://www.w3.org/2001/XMLSchema#base64Binary http://www.w3.org/2002/07/owl#sameAs http://www.w3.org/2001/XMLSchema#base64Binary
http://www.W3C.org/casaz/ontologies/2024/10/etvco/recognizedAspect http://www.w3.org/2002/07/owl#equivalentProperty http://www.W3C.org/casaz/ontologies/2024/10/etvco/recognizedAspect
http://www.W3C.org/casaz/ontologies/2024/10/etvco/dependsOn http://www.w3.org/2000/01/rdf-schema#domain http://www.

## 1. Story Title: Interpretations of Artifacts (sculptures, paintings, petroglyphs etc) with Unusual Features
CQ1: Which are the artifacts that are interpreted as depicting humanoid figures?

In [3]:
cq = """
PREFIX etvco:<http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX persp:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX dul:<http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX icon:<https://w3id.org/icon/ontology/>

SELECT ?workOfArt ?label
WHERE {
    ?realWorldObject a etvco:HumanoidFigure;
    icon:isFactualMeaningOf ?artisticMotif.
    ?artisticMotif icon:isRecognizedArtisticMotifOf ?preIconRec.
    ?preIconRec icon:aboutWorkOfArt ?workOfArt.
       ?workOfArt etvco:hasLabel ?label. 
}
"""

cqres = newg.query(cq)

for row in cqres:
    print(row)


(rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/img_1'), rdflib.term.Literal('Petroglyphs in caves in the mountains of Kimberley Australia'))


CQ2: What visual features(attribute) (e.g., head size, body shape) contribute to the alien interpretation? 

In [4]:
cq = """
PREFIX etvco:<http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX persp:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX dul:<http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX icon:<https://w3id.org/icon/ontology/>

SELECT DISTINCT ?character ?realWorldObject ?visualFeature ?quality
WHERE {
    ?character a etvco:AlienRelatedCharacter;
        icon:isCharacterOf ?image.
          ?image a icon:Image;
        icon:hasRecAttribute ?artisticMotif.
    ?artisticMotif a icon:ArtisticMotif;
        icon:hasFactualMeaning ?realWorldObject.
    ?realWorldObject a etvco:RealWorldObject.
      ?visualFeature dul:isPartOf ?realWorldObject;
        dul:hasQuality ?quality.  
}"""

cqres = newg.query(cq)

for row in cqres:
    print(row)

(rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#grey_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#humanoid_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#eye_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Large'))
(rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#grey_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#humanoid_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#head_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Oval'))


CQ3: "How do conspiracy theorists interpret certain artworks as depictions of aliens, and how are these interpretations evaluated or challenged by art historians and archaeologists?"

In [5]:
cq = """
PREFIX etvco:<http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX persp:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX dul:<http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX icon:<https://w3id.org/icon/ontology/>

SELECT DISTINCT ?workOfArt ?label ?alienCharacter ?character
WHERE {
    ?alienCharacter a etvco:AlienRelatedCharacter;
        icon:isCharacterOf ?image.
          ?image a icon:Image;
          icon:isImageOf ?iconRec.
    ?iconRec a icon:IconographicalRecognition;
      icon:aboutWorkOfArt ?workOfArt.
      ?workOfArt etvco:hasLabel ?label.
    OPTIONAL {
      ?character a icon:Character.
        FILTER NOT EXISTS {
          ?character a etvco:AlienRelatedCharacter.
        }
      ?character icon:isCharacterOf ?imageTwo.
      ?imageTwo icon:isImageOf ?iconRecTwo.
      ?iconRecTwo a icon:IconographicalRecognition;
      icon:aboutWorkOfArt ?workOfArt.
    }
}"""

cqres = newg.query(cq)

for row in cqres:
    print(row)

(rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/img_1'), rdflib.term.Literal('Petroglyphs in caves in the mountains of Kimberley Australia'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#grey_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#spirit_1'))


## 2. Story Title: Artifacts interpreted as evidence of Alien Technology
CQ1: "What kind of shapes are interpreted as alien technology?"

In [6]:
cq ="""
PREFIX etvco: <http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX dul:<http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?shape ?realworldobject
WHERE {
    ?shape a etvco:Shape.
    {
        ?shape dul:isPartOf ?composition.
        ?composition a etvco:CompositionOfAspects;
        etvco:makes ?object.
    }
    UNION
    {
        ?shape etvco:makes ?object .
    }
    ?object etvco:hasInterpretation ?interpretation.
    ?interpretation etvco:hasOutput ?realworldobject.
    ?realworldobject a etvco:AlienTechnologicalObject.
}
"""

cqres = newg.query(cq)

for row in cqres:
    print(row)

(rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Disc-shaped'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#ufo_1'))
(rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Round'), rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/ufo_2'))


CQ2: "How are the shapes interpreted by different agents?"

In [7]:
cq = """
PREFIX etvco:<http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX persp: <http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX dul:<http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX icon:<https://w3id.org/icon/ontology/>

SELECT DISTINCT ?agent ?shape ?interpretation ?realworldobject ?character ?place ?event ?namedObject ?symbol ?personification
WHERE {
  ?shape a etvco:Shape;
         etvco:makes ?object.
  ?object etvco:hasInterpretation ?interpretation.
  ?interpretation etvco:hasOutput ?realworldobject.
  
  OPTIONAL {
    ?realworldobject icon:isFactualMeaningOf ?artisticMotif.
    ?artisticMotif a icon:ArtisticMotif;
                   icon:isRecAttributeOf ?image.
    ?image a icon:Image.
    
    OPTIONAL {?image icon:hasCharacter ?character}
    OPTIONAL {?image icon:hasPlace ?place}
    OPTIONAL {?image icon:hasEvent ?event}
    OPTIONAL {?image icon:hasNamedObject ?namedObject}
    OPTIONAL {?image icon:hasSymbol ?symbol}
    OPTIONAL {?image icon:hasPersonification ?personification}
  }
  
  OPTIONAL {
    ?agent a dul:Agent;
      persp:creates ?interpretation.
  }
}
"""

cqres = newg.query(cq)

for row in cqres:
    print(row)

(rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/conspiracy_theorist_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Disc-shaped'), rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/interpretation_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#ufo_1'), None, None, None, None, None, None)
(rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#historian_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Dome-shaped'), rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/interpretation_2'), rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/unknown_1'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#angel_1'), None, None, None, None, None)


CQ3: "What artifacts are interpreted as depicting alien technology?"

In [8]:
cq = """
PREFIX etvco:<http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX icon:<https://w3id.org/icon/ontology/>
SELECT DISTINCT ?label ?workOfArt ?realworldobject
WHERE {
    ?realworldobject a etvco:AlienTechnologicalObject;
      icon:isFactualMeaningOf ?artisticMotif.
      ?artisticMotif icon:isRecognizedArtisticMotifOf ?preIconRec.
      ?preIconRec icon:aboutWorkOfArt ?workOfArt.
      ?workOfArt etvco:hasLabel ?label

}
"""

cqres = newg.query(cq)

for row in cqres:
    print(row)

(rdflib.term.Literal('Le Livre Des Bonnes Moeurs'), rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/img_3'), rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/ufo_2'))
(rdflib.term.Literal('The Crucifixion of Christ'), rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/img_2'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#ufo_1'))


CQ4: "What object position makes objects interpreted as alien technology?"

In [9]:
cq ="""
PREFIX etvco: <http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX dul:<http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?position ?realworldobject
WHERE {
   ?position a etvco:Position.
   {
   ?shape dul:isPartOf ?composition.
   ?composition a etvco:CompositionOfAspects;
      etvco:makes ?object.
   }
   UNION
   {
   ?position etvco:makes ?object .
     }
   ?object etvco:hasInterpretation ?interpretation.
   ?interpretation etvco:hasOutput ?realworldobject.
   ?realworldobject a etvco:AlienTechnologicalObject.
}
"""

cqres = newg.query(cq)

for row in cqres:
    print(row)


(rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#sky_1'), rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/ufo_2'))


## 3. Story title: Artifacts interpreted by art historians/archeologists as evidence of human culture/symbolism

## 4. Story title: Different periods, cultures and geographical locations and Interpretations

CQ1: Which are the period, place and cultural context of each artwork interpreted as depicting alien related concepts?

In [10]:
cq ="""
PREFIX etvco:<http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX persp: <http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX dul:<http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX icon:<https://w3id.org/icon/ontology/>

SELECT DISTINCT ?artwork ?label ?period ?place ?culturalContext
WHERE {
    ?artwork a icon:Artwork;
        etvco:hasLabel ?label .
    OPTIONAL
    { ?artwork etvco:wasCreatedInPeriod ?period }
    OPTIONAL
    { ?artwork etvco:wasCreatedInPlace ?place }
    OPTIONAL
    { ?artwork etvco:hasCulturalContext ?culturalContext }
    
    ?rec icon:aboutWorkOfArt ?artwork;
        persp:Cut ?interpretation.
    ?interpretation a etvco:AlienRelatedInterpretation.
}
"""

cqres = newg.query(cq)

for row in cqres:
    print(row)

(rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/img_1'), rdflib.term.Literal('Petroglyphs in caves in the mountains of Kimberley Australia'), None, rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Kimberley_australia'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#AboriginalTradition'))
(rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/img_2'), rdflib.term.Literal('The Crucifixion of Christ'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#17thCenturyMural'), rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/fred/domain.owl#Svetishoveli_cathedral'), None)


CQ2: Which is the period of time with the highest number of artworks interpreted as depicting alien related concepts?

In [ ]:
cq="""
PREFIX etvco:<http://www.W3C.org/casaz/ontologies/2024/10/etvco/>
PREFIX persp: <http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX dul:<http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX icon:<https://w3id.org/icon/ontology/>

SELECT ?periodClass (COUNT(?artwork) AS ?artworkCount)
WHERE {
    ?artwork a icon:Artwork.
    ?artwork etvco:wasCreatedInPeriod ?period .
    ?period a etvco:Period.
    ?period a ?periodClass .
    ?rec icon:aboutWorkOfArt ?artwork;
        persp:Cut ?interpretation.
    ?interpretation a etvco:AlienRelatedInterpretation.
}
GROUP BY ?periodClass
ORDER BY DESC(?artworkCount)
LIMIT 1 
"""


cqres = newg.query(cq)

for row in cqres:
    print(row)

(rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/ModernTime'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://www.W3C.org/casaz/ontologies/2024/10/etvco/Period'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Thing'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))


## 5. Story title: Cognitive perspectivisation in image recognition